In [1]:
# Cell 1: Import required libraries
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings
from llama_index.core.base.llms.types import ChatMessage, MessageRole, ImageBlock
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import SQLAutoVectorQueryEngine, RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.core import SQLDatabase
import chromadb
import sqlite3
import os

In [3]:
# Cell 2: Set up OpenAI API key
import openai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the OpenAI API key from the environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is loaded correctly
if openai.api_key:
    print("API key loaded successfully!")
else:
    print("API key not found. Please check your .env file.")

API key loaded successfully!


In [3]:
# Cell 3: Function to get table information
def get_table_info(db_path):
    """
    Get information about tables in the SQLite database.
    Returns a dictionary with table descriptions.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    table_info = {}
    for table in tables:
        table_name = table[0]
        # Get column information for each table
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        
        # Create description of table structure
        column_descriptions = [f"{col[1]} ({col[2]})" for col in columns]
        table_info[table_name] = {
            'columns': column_descriptions,
            'description': f"Table {table_name} contains columns: {', '.join(column_descriptions)}"
        }
    
    conn.close()
    return table_info

In [4]:
# Cell 4: Initialize components
# Define node parser and LLM
chunk_size = 1024
llm = OpenAI(temperature=0, model="gpt-4", streaming=True)

# Use Settings instead of ServiceContext
Settings.llm = llm
Settings.chunk_size = chunk_size

# Initialize components
text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser()

In [5]:
# Cell 5: Initialize Chroma
# Initialize Chroma
chroma_client = chromadb.PersistentClient(path="./vector_db")
chroma_collection = chroma_client.get_or_create_collection("multi_modal_rag")

# Create vector store
vector_store = ChromaVectorStore(
    chroma_collection=chroma_collection
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

In [6]:
# Cell 6: Set up vector store info and retriever
# Set up vector store info
vector_store_info = VectorStoreInfo(
    content_info="Database tables and their contents",
    metadata_info=[
        MetadataInfo(name="table_name", type="str", description="The name of the database table"),
    ],
)

# Create retriever
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

# Create retriever query engine
retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=Settings.llm, chunk_size=Settings.chunk_size
)

In [7]:
# # Cell 7: Create SQL database connection and query engine
# from llama_index.core import SQLDatabase
# from llama_index.core.query_engine import NLSQLTableQueryEngine

# # Define the path to your SQLite database
# sql_database_path = "./database/olist.sqlite"

# # Create SQL database connection
# sql_database = SQLDatabase.from_uri(
#     f"sqlite:///{sql_database_path}"
# )

# # Get table information
# table_info = get_table_info(sql_database_path)

# # Create SQL query engine
# sql_query_engine = NLSQLTableQueryEngine(
#     sql_database=sql_database,
#     tables=list(table_info.keys()),
# )

In [8]:
# Cell 7: Create SQL database connection and query engine
from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.objects import SQLTableSchema

# # Define the path to your SQLite database
# sql_database_path = "./database/olist.sqlite"

# Load environment variables from .env file
load_dotenv()
# Get SQL database path from environment variable
sql_database_path = os.getenv("SQL_DATABASE_PATH", "./database/olist.sqlite")

# Create SQL database connection
sql_database = SQLDatabase.from_uri(
    f"sqlite:///{sql_database_path}"
)

# Get table information
table_info = get_table_info(sql_database_path)

# Define table schemas with relationships
table_schemas = {}
for table_name in table_info.keys():
    # Get foreign key information
    conn = sqlite3.connect(sql_database_path)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA foreign_key_list({table_name});")
    foreign_keys = cursor.fetchall()
    
    # Create relationship context
    relationships = []
    for fk in foreign_keys:
        from_col = fk[3]
        to_table = fk[2]
        to_col = fk[4]
        relationships.append(f"The {from_col} in {table_name} is related to {to_col} in {to_table}")
    
    # Get column descriptions
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    column_descriptions = []
    for col in columns:
        col_name = col[1]
        col_type = col[2]
        column_descriptions.append(f"{col_name} ({col_type})")
    
    # Create table context
    context = f"Table {table_name} contains: {', '.join(column_descriptions)}"
    if relationships:
        context += f". Relationships: {'. '.join(relationships)}"
    
    table_schemas[table_name] = SQLTableSchema(
        table_name=table_name,
        context_str=context
    )
    
    conn.close()

# Create SQL query engine with enhanced capabilities
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=list(table_info.keys()),
    table_schemas=table_schemas,
    verbose=True
)

In [9]:
# Cell 8: Create tools and final query engine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.tools import QueryEngineTool

# Create tools
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for answering questions that require querying structured data from the database. "
        "This tool can analyze tables and their relationships to get specific data points. "
        f"Available tables: {', '.join(table_info.keys())}"
    )
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    name="vector_tool",
    description=(
        "Useful for answering semantic questions about the data that require understanding context "
        "and relationships beyond simple queries. This tool can provide high-level insights and "
        "analyze unstructured or complex information about the database contents."
    )
)

# Create selector with LLM
selector = LLMSingleSelector.from_defaults(llm=Settings.llm)

# Create the final multi-tool query engine
query_engine = RouterQueryEngine(
    selector=selector,
    query_engine_tools=[
        sql_tool,
        vector_tool
    ]
)

In [10]:
# Cell 8.5: Create enhanced query engine with multi-tool chaining capability
from typing import Optional, Dict, Any
from llama_index.core.base.response.schema import Response

class ChainedQueryEngine:
    def __init__(self, sql_engine, vector_engine, llm):
        self.sql_engine = sql_engine
        self.vector_engine = vector_engine
        self.llm = llm
        
    def _analyze_query_type(self, query: str) -> str:
        """Determine if the query needs SQL, vector search, or both."""
        prompt = f"""Analyze the following query and determine which type of processing it needs:
        Query: {query}
        
        Choose one of:
        1. "sql" - if it only needs structured database querying
        2. "vector" - if it only needs semantic search in vector database
        3. "chained" - if it needs both SQL first and then vector search
        4. "vector_then_sql" - if it needs vector search first and then SQL
        
        Return ONLY the type (sql/vector/chained/vector_then_sql) without explanation."""
        
        response = self.llm.complete(prompt)
        return response.text.strip().lower()
    
    def _extract_entity_from_sql_result(self, sql_response: Response, query: str) -> str:
        """Extract relevant entity from SQL response to use in vector search."""
        prompt = f"""From the SQL query result and the original question, extract the key entity needed for the next search.
        Original question: {query}
        SQL result: {sql_response}
        
        Return ONLY the entity name without any explanation or additional text."""
        
        response = self.llm.complete(prompt)
        return response.text.strip()
    
    def _combine_responses(self, sql_response: Optional[Response], vector_response: Optional[Response], query: str) -> Response:
        """Combine responses from both engines into a coherent answer."""
        prompt = f"""Combine the following information into a coherent answer:
        Original question: {query}
        SQL database result: {sql_response if sql_response else 'Not used'}
        Vector search result: {vector_response if vector_response else 'Not used'}
        
        Provide a natural, flowing response that answers the original question."""
        
        combined_response = self.llm.complete(prompt)
        
        # Create a new Response object with combined information
        response = Response(response=combined_response.text)
        if sql_response and hasattr(sql_response, 'metadata'):
            response.metadata = {'sql_metadata': sql_response.metadata}
        if vector_response and hasattr(vector_response, 'metadata'):
            response.metadata = {**(response.metadata or {}), 'vector_metadata': vector_response.metadata}
        
        # Combine source nodes if available
        source_nodes = []
        if sql_response and hasattr(sql_response, 'source_nodes'):
            source_nodes.extend(sql_response.source_nodes)
        if vector_response and hasattr(vector_response, 'source_nodes'):
            source_nodes.extend(vector_response.source_nodes)
        if source_nodes:
            response.source_nodes = source_nodes
            
        return response
    
    def _format_vector_query(self, original_query: str, sql_result: str) -> str:
        """Format a new query for vector search based on SQL results."""
        prompt = f"""Given the original question and SQL result, create a focused query for semantic search:
        Original question: {original_query}
        SQL result: {sql_result}
        
        Return ONLY the reformulated query without any explanation."""
        
        response = self.llm.complete(prompt)
        return response.text.strip()
    
    def query(self, query_str: str) -> Response:
        """Process a query using appropriate combination of SQL and vector search."""
        query_type = self._analyze_query_type(query_str)
        
        if query_type == "sql":
            return self.sql_engine.query(query_str)
        
        elif query_type == "vector":
            return self.vector_engine.query(query_str)
        
        elif query_type == "chained":
            # First get SQL result
            sql_response = self.sql_engine.query(query_str)
            
            # Extract relevant entity from SQL result
            entity = self._extract_entity_from_sql_result(sql_response, query_str)
            
            # Format new query for vector search
            vector_query = self._format_vector_query(query_str, str(sql_response))
            
            # Perform vector search
            vector_response = self.vector_engine.query(vector_query)
            
            # Combine responses
            return self._combine_responses(sql_response, vector_response, query_str)
        
        elif query_type == "vector_then_sql":
            # First get vector search result
            vector_response = self.vector_engine.query(query_str)
            
            # Format new query for SQL
            sql_query = self._format_vector_query(query_str, str(vector_response))
            
            # Perform SQL query
            sql_response = self.sql_engine.query(sql_query)
            
            # Combine responses
            return self._combine_responses(sql_response, vector_response, query_str)
        
        else:
            raise ValueError(f"Unknown query type: {query_type}")

# Create the enhanced query engine
enhanced_query_engine = ChainedQueryEngine(
    sql_engine=sql_query_engine,
    vector_engine=retriever_query_engine,
    llm=Settings.llm
)

# Update the query engine to use the enhanced version
query_engine = enhanced_query_engine

In [11]:
# # Cell 9: Example usage function
# def ask_question(question: str):
#     """
#     Ask a question about the database using both SQL and vector search capabilities.
    
#     Args:
#         question (str): The question to ask about the database
        
#     Returns:
#         str: The response from the query engine
#     """
#     try:
#         response = query_engine.query(question)
#         return response
#     except Exception as e:
#         return f"Error processing question: {str(e)}"

# # Example usage
# # You can uncomment and run these examples:
# """
# questions = [
#     "How many orders are there in total?",
#     "What is the distribution of order status?",
#     "What are the most common payment types?",
#     "Which sellers have the highest customer ratings?",
# ]

# for question in questions:
#     print(f"\nQuestion: {question}")
#     print(f"Answer: {ask_question(question)}")
# """

In [12]:
# Cell 9: Enhanced question answering with debug information
from typing import Any, List
from llama_index.core.base.response.schema import Response

def debug_query_engine_response(response: Response) -> None:
    """Print debug information from a query engine response."""
    print("\n=== Debug Information ===")
    
    # Get source nodes
    source_nodes = response.source_nodes
    if source_nodes:
        print("\nSource Nodes Used:")
        for idx, node in enumerate(source_nodes):
            print(f"\nSource {idx + 1}:")
            print(f"Text: {node.text[:200]}...")  # Print first 200 chars
            if node.metadata:
                print(f"Metadata: {node.metadata}")
            print(f"Score: {node.score if hasattr(node, 'score') else 'N/A'}")
    
    # Get tool used (if available)
    if hasattr(response, '_tool_used'):
        print(f"\nTool Used: {response._tool_used}")
    
    # Print any SQL queries that were executed
    if hasattr(response, 'metadata') and response.metadata:
        if 'sql_query' in response.metadata:
            print(f"\nSQL Query Executed:")
            print(response.metadata['sql_query'])
        if 'sql_response' in response.metadata:
            print(f"\nSQL Response:")
            print(response.metadata['sql_response'])

def ask_question(question: str, debug: bool = True):
    """
    Ask a question about the database using both SQL and vector search capabilities.
    
    Args:
        question (str): The question to ask about the database
        debug (bool): Whether to show debug information
        
    Returns:
        str: The response from the query engine
    """
    try:
        print(f"\n🤔 Question: {question}")
        
        # Get the response from the query engine
        response = query_engine.query(question)
        
        print(f"\n📝 Answer: {response}")
        
        if debug:
            debug_query_engine_response(response)
            
        return response
    except Exception as e:
        return f"Error processing question: {str(e)}"


In [14]:
# response = ask_question("Explain multi head attention?")
# print(response)

In [15]:
# response = ask_question("what are the top 5 selling products by price?")
# print(response)


In [16]:
# response = ask_question("""
#     Who are the top 5 sellers based on a combination of:
#     - Total order volume
#     - Average customer rating
#     - Average delivery time
#     Provide their performance metrics.
# """)
# print(response)


In [17]:
# response = ask_question("What is the population of Tokyo?")
# print(response)

In [23]:
response = ask_question("history of the city tokyo?")
print(response)


🤔 Question: history of the city tokyo?

📝 Answer: Tokyo started as a small fishing village named Edo in the old Musashi Province. The Edo clan built walls to protect the town in the late 12th century, and in 1457, Ōta Dōkan built Edo Castle. In 1590, Tokugawa Ieyasu made Edo his base and became shogun in 1603, making the town the center of his military government. This marked the beginning of the Edo period, during which Edo grew into one of the largest cities in the world. 

Although Edo was not the capital of Japan, it held more power due to the shogun's influence. The Meiji Restoration in 1869 removed the shogun from power, and the 17-year-old Emperor Meiji moved to Edo, transforming the old Edo Castle into the Kokyo, the Imperial Palace. 

In the same year, the Tokyo Prefecture and the city of Tokyo were established, serving as the capital city until 1943. In 1943, Tokyo City and the associated municipalities of what was formerly Tokyo Prefecture combined into one. 

Tokyo suffere

In [24]:
response.response

"Tokyo started as a small fishing village named Edo in the old Musashi Province. The Edo clan built walls to protect the town in the late 12th century, and in 1457, Ōta Dōkan built Edo Castle. In 1590, Tokugawa Ieyasu made Edo his base and became shogun in 1603, making the town the center of his military government. This marked the beginning of the Edo period, during which Edo grew into one of the largest cities in the world. \n\nAlthough Edo was not the capital of Japan, it held more power due to the shogun's influence. The Meiji Restoration in 1869 removed the shogun from power, and the 17-year-old Emperor Meiji moved to Edo, transforming the old Edo Castle into the Kokyo, the Imperial Palace. \n\nIn the same year, the Tokyo Prefecture and the city of Tokyo were established, serving as the capital city until 1943. In 1943, Tokyo City and the associated municipalities of what was formerly Tokyo Prefecture combined into one. \n\nTokyo suffered two major catastrophes in the 20th century

In [20]:
response.metadata

{'sql_metadata': {'45daa04a-2fe6-4c22-9104-766a9ce77843': {'sql_query': 'SELECT product_id, price \nFROM order_items \nORDER BY price DESC \nLIMIT 5;',
   'result': [('489ae2aa008f021502940f251d4cce7f', 6735.0),
    ('69c590f7ffc7bf8db97190b6cb6ed62e', 6729.0),
    ('1bdf5e6731585cf01aa8169c7028d6ad', 6499.0),
    ('a6492cc69376c469ab6f61d8f44de961', 4799.0),
    ('c3ed642d592594bb648ff4a04cee2747', 4690.0)],
   'col_keys': ['product_id', 'price']},
  'sql_query': 'SELECT product_id, price \nFROM order_items \nORDER BY price DESC \nLIMIT 5;',
  'result': [('489ae2aa008f021502940f251d4cce7f', 6735.0),
   ('69c590f7ffc7bf8db97190b6cb6ed62e', 6729.0),
   ('1bdf5e6731585cf01aa8169c7028d6ad', 6499.0),
   ('a6492cc69376c469ab6f61d8f44de961', 4799.0),
   ('c3ed642d592594bb648ff4a04cee2747', 4690.0)],
  'col_keys': ['product_id', 'price']},
 'vector_metadata': {'95e0ba06-eb60-4b1d-8fe3-2c5a5b01cbb2': {'doc_id': '4293812d-1b80-46b4-ac4a-e2e1d4e128e8'},
  'd72c8178-4ae3-4aa9-9832-94906005dc0f': 